# Scikit-learn - Evaluación de Modelos

Este notebook cubre las técnicas de evaluación de modelos en scikit-learn: métricas de clasificación, regresión, validación cruzada y ajuste de hiperparámetros.

## Importar Módulos


In [1]:
from sklearn import datasets
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate
from sklearn.model_selection import KFold, StratifiedKFold, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, roc_curve
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import matplotlib.pyplot as plt


## Métricas de Clasificación


In [2]:
# Cargar datos y entrenar modelo
X, y = datasets.load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LogisticRegression(random_state=42, max_iter=1000)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)

# Métricas básicas
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print("Métricas de Clasificación:")
print(f"Accuracy (Precisión): {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall (Sensibilidad): {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

# Matriz de confusión
cm = confusion_matrix(y_test, y_pred)
print(f"\nMatriz de Confusión:\n{cm}")

# Reporte completo
print("\nReporte de Clasificación:")
print(classification_report(y_test, y_pred))


Métricas de Clasificación:
Accuracy (Precisión): 1.0000
Precision: 1.0000
Recall (Sensibilidad): 1.0000
F1-Score: 1.0000

Matriz de Confusión:
[[10  0  0]
 [ 0  9  0]
 [ 0  0 11]]

Reporte de Clasificación:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      1.00      1.00         9
           2       1.00      1.00      1.00        11

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30



## Métricas de Regresión


In [3]:
from sklearn.linear_model import LinearRegression

# Cargar datos de regresión
X, y = datasets.load_diabetes(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model_reg = LinearRegression()
model_reg.fit(X_train, y_train)
y_pred_reg = model_reg.predict(X_test)

# Métricas de regresión
mse = mean_squared_error(y_test, y_pred_reg)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred_reg)
r2 = r2_score(y_test, y_pred_reg)

print("Métricas de Regresión:")
print(f"MSE (Mean Squared Error): {mse:.4f}")
print(f"RMSE (Root Mean Squared Error): {rmse:.4f}")
print(f"MAE (Mean Absolute Error): {mae:.4f}")
print(f"R² Score: {r2:.4f}")


Métricas de Regresión:
MSE (Mean Squared Error): 2900.1936
RMSE (Root Mean Squared Error): 53.8534
MAE (Mean Absolute Error): 42.7941
R² Score: 0.4526


## Validación Cruzada


In [4]:
# Validación cruzada simple
X, y = datasets.load_iris(return_X_y=True)
model = LogisticRegression(random_state=42, max_iter=1000)

# K-Fold Cross Validation
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = cross_val_score(model, X, y, cv=kfold, scoring='accuracy')
print("K-Fold Cross Validation (5 folds):")
print(f"Scores: {cv_scores}")
print(f"Media: {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")

# Stratified K-Fold (mantiene proporción de clases)
skfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_scores_strat = cross_val_score(model, X, y, cv=skfold, scoring='accuracy')
print(f"\nStratified K-Fold Cross Validation:")
print(f"Scores: {cv_scores_strat}")
print(f"Media: {cv_scores_strat.mean():.4f} (+/- {cv_scores_strat.std() * 2:.4f})")

# Múltiples métricas
cv_results = cross_validate(model, X, y, cv=5, scoring=['accuracy', 'precision_weighted', 'recall_weighted', 'f1_weighted'])
print(f"\nMúltiples métricas:")
for metric in cv_results.keys():
    if metric.startswith('test_'):
        print(f"  {metric}: {cv_results[metric].mean():.4f}")


K-Fold Cross Validation (5 folds):
Scores: [1.         1.         0.93333333 0.96666667 0.96666667]
Media: 0.9733 (+/- 0.0499)

Stratified K-Fold Cross Validation:
Scores: [1.         0.96666667 0.93333333 1.         0.93333333]
Media: 0.9667 (+/- 0.0596)



Múltiples métricas:
  test_accuracy: 0.9733
  test_precision_weighted: 0.9768
  test_recall_weighted: 0.9733
  test_f1_weighted: 0.9732


## Búsqueda de Hiperparámetros


In [5]:
# Grid Search (búsqueda exhaustiva)
X, y = datasets.load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Definir parámetros a buscar
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7, None],
    'min_samples_split': [2, 5, 10]
}

rf = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(rf, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

print("Grid Search:")
print(f"Mejores parámetros: {grid_search.best_params_}")
print(f"Mejor score (CV): {grid_search.best_score_:.4f}")
print(f"Score en test: {grid_search.score(X_test, y_test):.4f}")

# Randomized Search (búsqueda aleatoria, más eficiente)
from scipy.stats import randint
param_dist = {
    'n_estimators': randint(50, 300),
    'max_depth': [3, 5, 7, None],
    'min_samples_split': randint(2, 20)
}

random_search = RandomizedSearchCV(rf, param_dist, n_iter=20, cv=5, scoring='accuracy', random_state=42, n_jobs=-1)
random_search.fit(X_train, y_train)

print(f"\nRandomized Search:")
print(f"Mejores parámetros: {random_search.best_params_}")
print(f"Mejor score (CV): {random_search.best_score_:.4f}")
print(f"Score en test: {random_search.score(X_test, y_test):.4f}")


Grid Search:
Mejores parámetros: {'max_depth': 3, 'min_samples_split': 2, 'n_estimators': 50}
Mejor score (CV): 0.9500
Score en test: 1.0000



Randomized Search:
Mejores parámetros: {'max_depth': 7, 'min_samples_split': 16, 'n_estimators': 156}
Mejor score (CV): 0.9500
Score en test: 1.0000


## Curvas ROC y AUC (Clasificación Binaria)


In [6]:
# Para clasificación binaria
X, y = datasets.load_breast_cancer(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model_binary = LogisticRegression(random_state=42, max_iter=1000)
model_binary.fit(X_train, y_train)
y_pred_proba_binary = model_binary.predict_proba(X_test)[:, 1]

# Calcular ROC curve y AUC
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba_binary)
auc_score = roc_auc_score(y_test, y_pred_proba_binary)

print(f"AUC Score: {auc_score:.4f}")
print("Nota: Para visualizar la curva ROC, usar:")
print("  plt.plot(fpr, tpr, label=f'AUC = {auc_score:.2f}')")
print("  plt.plot([0, 1], [0, 1], 'k--')")
print("  plt.xlabel('False Positive Rate')")
print("  plt.ylabel('True Positive Rate')")
print("  plt.title('ROC Curve')")
print("  plt.legend()")
print("  plt.show()")


AUC Score: 0.9977
Nota: Para visualizar la curva ROC, usar:
  plt.plot(fpr, tpr, label=f'AUC = {auc_score:.2f}')
  plt.plot([0, 1], [0, 1], 'k--')
  plt.xlabel('False Positive Rate')
  plt.ylabel('True Positive Rate')
  plt.title('ROC Curve')
  plt.legend()
  plt.show()


/home/sergio/Documentos/Curso IA y Big Data/cheat-sheets-ia/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 1000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=1000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
